In [398]:
!pip install tmdbv3api

In [399]:
import pandas as pd
import requests
import json

from tmdbv3api import Movie, TMDb

#1º Parte: Coleta de Dados
Nessa parte iremos buscar os filmes atualizados do IMDB e, em seguida, iremos passá-los para um arquivo CSV.

In [400]:
tmdb = TMDb()
tmdb.api_key = "6c7fe2d7eeb02195b49f11e07a02fb2d"
tmdb_movie = Movie()


## Compreendendo melhor a estrutura do dataset:

In [401]:
df = pd.read_csv("movies_metadata.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [402]:
#buscando o último ano em que o dataset foi atualizado:
years = df['release_date'].dropna()
years_list = []

for year in years:
  year = str(year).split('-')
  year = year[0]
  years_list.append(int(year))
  
print(max(years_list))

2020


Portanto, precisamos atualizar o dataset de filmes, visto que temos um gap de 2 anos de filmes novos.

## Filmes de 2020:

In [403]:
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"


A lista de filmes de 2020 funciona por trimestres, ou seja, para o ano inteiro, precisamos pegar 4 trimestres e fazer quatro dataframes para juntá-los. 

In [404]:
df1_2020 = pd.read_html(url, header=0)[2]
df2_2020 = pd.read_html(url, header=0)[3]
df3_2020 = pd.read_html(url, header=0)[4]
df4_2020 = pd.read_html(url, header=0)[5]

In [405]:
df1.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]


In [406]:
df_2020 = df1_2020.append(df2_2020.append(df3_2020.append(df4_2020, ignore_index=True), ignore_index=True), ignore_index=True)  

In [407]:
df_2020["Opening"]

0       JANUARY
1       JANUARY
2       JANUARY
3       JANUARY
4       JANUARY
         ...   
268    DECEMBER
269    DECEMBER
270    DECEMBER
271    DECEMBER
272    DECEMBER
Name: Opening, Length: 273, dtype: object

In [408]:
df_2020 = df_2020.filter(["Title", "Opening"], axis=1)

In [409]:
df_2020.shape

(273, 2)

In [410]:
type(df_2020)

pandas.core.frame.DataFrame

In [411]:
def get_id(title):
  result = tmdb_movie.search(title)
  #Tem um erro na lista de filmes da imdb, tem um filme vazio que foi deletado e não apagaram. Para isso, devo tratar o Erro:
  if result != []:
    movie_id = result[0].id
    return movie_id

In [412]:
df_2020["id"] = df_2020["Title"].map(lambda x: get_id(str(x)))

In [413]:
df_2020.head()

,Title,Opening,id
0,The Grudge,JANUARY,1970.0
1,Underwater,JANUARY,443791.0
2,Like a Boss,JANUARY,526019.0
3,Three Christs,JANUARY,418533.0
4,Inherit the Viper,JANUARY,634904.0


In [414]:
df_2020.drop(columns=["Opening"], inplace=True)

In [415]:
df_2020.rename(columns = {"Title": "original_title"}, inplace= True)

In [416]:
df_2020.head()

,original_title,id
0,The Grudge,1970.0
1,Underwater,443791.0
2,Like a Boss,526019.0
3,Three Christs,418533.0
4,Inherit the Viper,634904.0


##Filmes de 2021:

In [417]:
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [418]:
df1_2021 = pd.read_html(url, header=0)[2]
df2_2021 = pd.read_html(url, header=0)[3]
df3_2021 = pd.read_html(url, header=0)[4]
df4_2021 = pd.read_html(url, header=0)[5]

In [419]:
df_2021 = df1_2021.append(df2_2021.append(df3_2021.append(df4_2021, ignore_index=True), ignore_index=True), ignore_index=True)  

In [420]:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 11 columns):
 #   Column                                                                       Non-Null Count  Dtype  
---  ------                                                                       --------------  -----  
 0   Rank                                                                         15 non-null     float64
 1   Title                                                                        278 non-null    object 
 2   Distributor                                                                  15 non-null     object 
 3   Domestic gross                                                               15 non-null     object 
 4   International gross                                                          15 non-null     object 
 5   Opening                                                                      263 non-null    object 
 6   Opening.1                                 

In [421]:
df_2021.tail()

,Rank,Title,Distributor,Domestic gross,International gross,Opening,Opening.1,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
273,NaN,My Little Pony: A New Generation,NaN,NaN,NaN,SEPTEMBER,24.0,Netflix / Entertainment One / Boulder Media,"Robert Cullen, Jose Ucha (directors); Gillian ...",NaN,[195]
274,NaN,Dear Evan Hansen,NaN,NaN,NaN,SEPTEMBER,24.0,Universal Pictures / Perfect World Pictures,Stephen Chbosky (director); Steven Levenson (s...,NaN,[196]
275,NaN,The Guilty,NaN,NaN,NaN,SEPTEMBER,24.0,Netflix / Bold Films / Nine Stories Productions,Antoine Fuqua (director); Nic Pizzolatto (scre...,NaN,[167]
276,NaN,Birds of Paradise,NaN,NaN,NaN,SEPTEMBER,24.0,Amazon Studios,Sarah Adina Smith (director/screenplay); Krist...,NaN,[197]
277,NaN,After We Fell,NaN,NaN,NaN,SEPTEMBER,30.0,Voltage Pictures,Castille Landon (director); Sharon Soboil (scr...,NaN,[198]


In [422]:
df_2021.shape

(278, 11)

In [423]:
df_2021 = df_2021.filter(["Title", "Opening"], axis=1)

In [424]:
df_2021["id"] = df_2021["Title"].map(lambda x: get_id(str(x)))

In [425]:
df_2021.head()

,Title,Opening,id
0,Spider-Man: No Way Home*,NaN,634649.0
1,Shang-Chi and the Legend of the Ten Rings,NaN,566525.0
2,Venom: Let There Be Carnage,NaN,580489.0
3,Black Widow,NaN,497698.0
4,F9,NaN,385128.0


In [426]:
df_2021.drop(columns=["Opening"], inplace=True)

In [427]:
df_2021.head()

,Title,id
0,Spider-Man: No Way Home*,634649.0
1,Shang-Chi and the Legend of the Ten Rings,566525.0
2,Venom: Let There Be Carnage,580489.0
3,Black Widow,497698.0
4,F9,385128.0


In [428]:
df_2021.rename(columns = {"Title": "original_title"}, inplace= True)

In [429]:
df_2021.head()

,original_title,id
0,Spider-Man: No Way Home*,634649.0
1,Shang-Chi and the Legend of the Ten Rings,566525.0
2,Venom: Let There Be Carnage,580489.0
3,Black Widow,497698.0
4,F9,385128.0


##Concatenando os Dataframes:

In [430]:
listas_dfs = [df_2020, df_2021]

In [431]:
dfs_concat = pd.concat(listas_dfs)

In [432]:
dfs_concat.head()

,original_title,id
0,The Grudge,1970.0
1,Underwater,443791.0
2,Like a Boss,526019.0
3,Three Christs,418533.0
4,Inherit the Viper,634904.0


In [433]:
dfs_concat.tail()

,original_title,id
273,My Little Pony: A New Generation,597316.0
274,Dear Evan Hansen,567690.0
275,The Guilty,567748.0
276,Birds of Paradise,667969.0
277,After We Fell,744275.0


In [434]:
dfs_concat

,original_title,id
0,The Grudge,1970.0
1,Underwater,443791.0
2,Like a Boss,526019.0
3,Three Christs,418533.0
4,Inherit the Viper,634904.0
...,...,...
273,My Little Pony: A New Generation,597316.0
274,Dear Evan Hansen,567690.0
275,The Guilty,567748.0
276,Birds of Paradise,667969.0


In [435]:
dfs_concat = dfs_concat.dropna()

In [436]:
dfs_concat.tail()

,original_title,id
273,My Little Pony: A New Generation,597316.0
274,Dear Evan Hansen,567690.0
275,The Guilty,567748.0
276,Birds of Paradise,667969.0
277,After We Fell,744275.0


In [437]:
dfs_concat.shape

(549, 2)

In [438]:
df_2020.shape

(273, 2)

In [439]:
df_2021.shape

(278, 2)

##Conjunto de Dados do Kaggle:

In [440]:
df_movies = pd.read_csv("tmdb_5000_movies.csv")

In [441]:
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


##Retorno JSON:

Iremos pegar todas as informações do JSON que batem com o dataset fornecido pelo Kaggle:

In [442]:
response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(406563, tmdb.api_key))

In [443]:
data_json = response.json()
data_json

{'adult': False,
 'backdrop_path': '/PwI3EfasE9fVuXsmMu9ffJh0Re.jpg',
 'belongs_to_collection': {'backdrop_path': '/5FrPZHgbbmTIq0oxpwSGqu5HyXC.jpg',
  'id': 228446,
  'name': 'Insidious Collection',
  'poster_path': '/w1213HKk1XKSwHiBgjkWghn9biC.jpg'},
 'budget': 10000000,
 'genres': [{'id': 27, 'name': 'Horror'},
  {'id': 9648, 'name': 'Mystery'},
  {'id': 53, 'name': 'Thriller'}],
 'homepage': 'http://www.insidiousmovie.com',
 'id': 406563,
 'imdb_id': 'tt5726086',
 'original_language': 'en',
 'original_title': 'Insidious: The Last Key',
 'overview': 'Parapsychologist Elise Rainier and her team travel to Five Keys, NM, to investigate a man’s claim of a haunting. Terror soon strikes when Rainier realizes that the house he lives in was her family’s old home.',
 'popularity': 56.071,
 'poster_path': '/nb9fc9INMg8kQ8L7sE7XTNsZnUX.jpg',
 'production_companies': [{'id': 11341,
   'logo_path': '/xytTBODEy3p20ksHL4Ftxr483Iv.png',
   'name': 'Stage 6 Films',
   'origin_country': 'US'},
  {'i

##Gênero dos filmes:

In [444]:
dfs_concat.head()

,original_title,id
0,The Grudge,1970.0
1,Underwater,443791.0
2,Like a Boss,526019.0
3,Three Christs,418533.0
4,Inherit the Viper,634904.0


In [445]:
def get_genres(movie_id):
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["genres"]

In [446]:
dfs_concat['genres'] = dfs_concat["id"].map(lambda x: get_genres(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [447]:
dfs_concat.head()

,original_title,id,genres
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n..."
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam..."
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]"
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]"
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name..."


##Budget dos filmes:

In [448]:
def get_budget(movie_id):
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["budget"]

In [449]:
dfs_concat['budget'] = dfs_concat["id"].map(lambda x: get_budget(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [450]:
dfs_concat.head()

,original_title,id,genres,budget
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0


##Palavras-chaves (Keywords):

In [451]:
response = requests.get("https://api.themoviedb.org/3/movie/{}/keywords?api_key={}". format(406563, tmdb.api_key))

In [452]:
data_json = response.json()
data_json

{'id': 406563,
 'keywords': [{'id': 2723, 'name': 'medium'},
  {'id': 2849, 'name': 'key'},
  {'id': 3358, 'name': 'haunted house'},
  {'id': 9675, 'name': 'prequel'},
  {'id': 13153, 'name': 'spirit'},
  {'id': 208611, 'name': '1950s'}]}

In [453]:
def get_keywords(movie_id):
  response = requests.get("https://api.themoviedb.org/3/movie/{}/keywords?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["keywords"]

In [454]:
dfs_concat['keywords'] = dfs_concat['id'].map(lambda x: get_keywords(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [455]:
dfs_concat.head()

,original_title,id,genres,budget,keywords
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n..."
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79..."
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18..."
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[]
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24..."


##Original Language:

In [456]:
def get_original_language(movie_id):
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["original_language"]

In [457]:
dfs_concat['original_language'] = dfs_concat['id'].map(lambda x: get_original_language(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [458]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en


## Overview:


In [459]:
#Pegando a sinopse de cada filme
#Caso desejemos mudar a língua, basta acrescentar "&language=pt-br" após a api_key.
def get_overview(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["overview"]

In [460]:
dfs_concat['overview'] = dfs_concat['id'].map(lambda x: get_overview(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [461]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib..."


##Popularity:

In [462]:
def get_popularity(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["popularity"]

In [463]:
dfs_concat['popularity'] = dfs_concat['id'].map(lambda x: get_popularity(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [464]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569


##Production_Company:

In [465]:
def get_production_companies(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["production_companies"]

In [466]:
dfs_concat['production_companies'] = dfs_concat['id'].map(lambda x: get_production_companies(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [467]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy..."
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih..."
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar..."
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro..."
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar..."


##Production_countries:

In [468]:
def get_production_countries(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["production_countries"]

In [469]:
dfs_concat['production_countries'] = dfs_concat['id'].map(lambda x: get_production_countries(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [470]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_..."
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o..."
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o..."
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o..."


##Release_date

In [471]:
def get_release_date(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["release_date"]

In [472]:
dfs_concat['release_date'] = dfs_concat['id'].map(lambda x: get_release_date(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [473]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28


##Revenue (Lucro):

In [474]:
def get_revenue(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["revenue"]

In [475]:
dfs_concat['revenue'] = dfs_concat['id'].map(lambda x: get_revenue(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [476]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0


##Runtime

In [477]:
def get_runtime(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["runtime"]

In [478]:
dfs_concat['runtime'] = dfs_concat['id'].map(lambda x: get_runtime(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [479]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90


## Línguas faladas (Spoken_languages):

In [480]:
def get_spoken_languages(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["spoken_languages"]

In [481]:
dfs_concat['spoken_languages'] = dfs_concat['id'].map(lambda x: get_spoken_languages(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [482]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'..."
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'..."
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'..."
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'..."
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'..."


##Satus:


In [483]:
def get_status(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["status"]

In [484]:
dfs_concat['status'] = dfs_concat['id'].map(lambda x: get_status(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [485]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released


##Tagline:

In [486]:
def get_tagline(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["tagline"]

In [487]:
dfs_concat['tagline'] = dfs_concat['id'].map(lambda x: get_tagline(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [488]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It never forgives. It never forgets.
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7 miles below the ocean surface something has ...
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The world of beauty is about to get ugly.
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some debts can't be paid.


## Tittle:

In [489]:
def get_title(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["title"]

In [490]:
dfs_concat['title'] = dfs_concat['id'].map(lambda x: get_title(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [491]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It never forgives. It never forgets.,The Grudge
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7 miles below the ocean surface something has ...,Underwater
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The world of beauty is about to get ugly.,Like a Boss
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Three Christs
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some debts can't be paid.,Inherit the Viper


##Vote_average:	

In [492]:
def get_vote_average(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["vote_average"]

In [493]:
dfs_concat['vote_average'] = dfs_concat['id'].map(lambda x: get_vote_average(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [494]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It never forgives. It never forgets.,The Grudge,5.9
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7 miles below the ocean surface something has ...,Underwater,6.3
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The world of beauty is about to get ugly.,Like a Boss,6.4
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Three Christs,6.8
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some debts can't be paid.,Inherit the Viper,5.8


##Vote_count:

In [495]:
def get_vote_count(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["vote_count"]

In [496]:
dfs_concat['vote_count'] = dfs_concat['id'].map(lambda x: get_vote_count(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [497]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It never forgives. It never forgets.,The Grudge,5.9,2323
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7 miles below the ocean surface something has ...,Underwater,6.3,2320
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The world of beauty is about to get ugly.,Like a Boss,6.4,532
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Three Christs,6.8,70
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some debts can't be paid.,Inherit the Viper,5.8,37


## Homepage:



In [498]:
def get_homepage(movie_id):
  response = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["homepage"]

In [499]:
dfs_concat['homepage'] = dfs_concat["id"].map(lambda x: get_homepage(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [500]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,homepage
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It never forgives. It never forgets.,The Grudge,5.9,2323,
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7 miles below the ocean surface something has ...,Underwater,6.3,2320,https://www.20thcenturystudios.com/movies/unde...
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The world of beauty is about to get ugly.,Like a Boss,6.4,532,https://www.paramountmovies.com/movies/like-a-...
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Three Christs,6.8,70,
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some debts can't be paid.,Inherit the Viper,5.8,37,


##Comparando os DataFrames:

In [501]:
df_movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [502]:
df_movies.shape

(4803, 20)

In [503]:
dfs_concat.columns

Index(['original_title', 'id', 'genres', 'budget', 'keywords',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'homepage'],
      dtype='object')

In [504]:
dfs_concat.shape

(549, 20)

##Inserção dos créditos:

In [505]:
df_movies_credits = pd.read_csv("tmdb_5000_credits.csv")

In [506]:
df_movies_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [507]:
df_movies_credits.rename(columns = {'movie_id': "id"}, inplace=True)

In [508]:
df_movies_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4803 non-null   int64 
 1   title   4803 non-null   object
 2   cast    4803 non-null   object
 3   crew    4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [509]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [510]:
df_movies = df_movies.merge(df_movies_credits, on="id")

In [511]:
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


##Inserindo os créditos dos filmes de 2020 e 2021:

In [512]:
def get_credits_cast(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}/credits?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["cast"]

In [513]:
dfs_concat['cast'] = dfs_concat['id'].map(lambda x: get_credits_cast(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [514]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,homepage,cast
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It never forgives. It never forgets.,The Grudge,5.9,2323,,"[{'adult': False, 'gender': 1, 'id': 11863, 'k..."
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7 miles below the ocean surface something has ...,Underwater,6.3,2320,https://www.20thcenturystudios.com/movies/unde...,"[{'adult': False, 'gender': 1, 'id': 37917, 'k..."
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The world of beauty is about to get ugly.,Like a Boss,6.4,532,https://www.paramountmovies.com/movies/like-a-...,"[{'adult': False, 'gender': 1, 'id': 1230868, ..."
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Three Christs,6.8,70,,"[{'adult': False, 'gender': 2, 'id': 1205, 'kn..."
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some debts can't be paid.,Inherit the Viper,5.8,37,,"[{'adult': False, 'gender': 2, 'id': 2299, 'kn..."


In [515]:
def get_credits_crew(movie_id): 
  response = requests.get("https://api.themoviedb.org/3/movie/{}/credits?api_key={}". format(movie_id, tmdb.api_key))
  data_json = response.json()
  return data_json["crew"]

In [516]:
dfs_concat['crew'] = dfs_concat['id'].map(lambda x: get_credits_crew(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [517]:
dfs_concat.head()

,original_title,id,genres,budget,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,homepage,cast,crew
0,The Grudge,1970.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",10000000,"[{'id': 428, 'name': 'nurse'}, {'id': 6095, 'n...",en,An American nurse living and working in Tokyo ...,43.412,"[{'id': 829, 'logo_path': '/mzpAmEQ5P1gFvdRoNy...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",2004-10-22,183474602,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,It never forgives. It never forgets.,The Grudge,5.9,2323,,"[{'adult': False, 'gender': 1, 'id': 11863, 'k...","[{'adult': False, 'gender': 2, 'id': 7623, 'kn..."
1,Underwater,443791.0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",50000000,"[{'id': 3521, 'name': 'earthquake'}, {'id': 79...",en,After an earthquake destroys their underwater ...,43.507,"[{'id': 7076, 'logo_path': '/8BFxn9NUWSgp0ndih...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-08,40882928,95,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,7 miles below the ocean surface something has ...,Underwater,6.3,2320,https://www.20thcenturystudios.com/movies/unde...,"[{'adult': False, 'gender': 1, 'id': 37917, 'k...","[{'adult': False, 'gender': 2, 'id': 2122, 'kn..."
2,Like a Boss,526019.0,"[{'id': 35, 'name': 'Comedy'}]",29000000,"[{'id': 160483, 'name': 'investor'}, {'id': 18...",en,Two female friends with very different ideals ...,16.741,"[{'id': 106497, 'logo_path': None, 'name': 'Ar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2020-01-09,26925979,83,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The world of beauty is about to get ugly.,Like a Boss,6.4,532,https://www.paramountmovies.com/movies/like-a-...,"[{'adult': False, 'gender': 1, 'id': 1230868, ...","[{'adult': False, 'gender': 2, 'id': 23486, 'k..."
3,Three Christs,418533.0,"[{'id': 18, 'name': 'Drama'}]",0,[],en,Dr. Alan Stone breaks new ground for treatment...,10.047,"[{'id': 35000, 'logo_path': None, 'name': 'Bro...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-09-23,0,117,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Three Christs,6.8,70,,"[{'adult': False, 'gender': 2, 'id': 1205, 'kn...","[{'adult': False, 'gender': 2, 'id': 18250, 'k..."
4,Inherit the Viper,634904.0,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",0,"[{'id': 1415, 'name': 'small town'}, {'id': 24...",en,"Since the death of their father, the Riley sib...",10.569,"[{'id': 22815, 'logo_path': None, 'name': 'Bar...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-09-28,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some debts can't be paid.,Inherit the Viper,5.8,37,,"[{'adult': False, 'gender': 2, 'id': 2299, 'kn...","[{'adult': False, 'gender': 2, 'id': 115090, '..."


##Concatenação Final:

In [518]:
df_movies.drop(['title_y'], axis = 1, inplace=True)

In [519]:
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [520]:
df_movies.rename(columns={"title_x": "title"}, inplace=True)

In [521]:
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [522]:
dataframes = [df_movies, dfs_concat]

In [523]:
df_movies.shape

(4803, 22)

In [524]:
dfs_concat.shape

(549, 22)

In [525]:
df_final = pd.concat(dataframes)

In [526]:
df_final.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995.0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285.0,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647.0,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026.0,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529.0,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [527]:
df_movies.shape

(4803, 22)

In [528]:
dfs_concat.shape

(549, 22)

In [529]:
df_final.shape

(5352, 22)

##Salvar em CSV:

In [531]:
df_final.to_csv(r'/content/df_final.csv', index=False, header=True)

In [532]:
df_test = pd.read_csv("df_final.csv")

In [534]:
df_test.tail()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
5347,0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",NaN,597316.0,"[{'id': 3987, 'name': 'flying horse'}, {'id': ...",en,My Little Pony: A New Generation,Equestria's divided. But a bright-eyed hero be...,173.245,"[{'id': 57031, 'logo_path': None, 'name': 'All...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",2021-09-23,0,91.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Ride or Fly,My Little Pony: A New Generation,8.1,175,"[{'adult': False, 'gender': 1, 'id': 67599, 'k...","[{'adult': False, 'gender': 2, 'id': 20773, 'k..."
5348,28000000,"[{'id': 18, 'name': 'Drama'}]",https://www.dehmovie.com,567690.0,"[{'id': 236, 'name': 'suicide'}, {'id': 1646, ...",en,Dear Evan Hansen,"Evan Hansen, a high schooler with social anxie...",79.207,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'CN', 'name': 'China'}, {'iso_...",2021-09-24,15732069,137.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,An honest and original story for the outsider ...,Dear Evan Hansen,6.5,124,"[{'adult': False, 'gender': 2, 'id': 1107296, ...","[{'adult': False, 'gender': 2, 'id': 1638, 'kn..."
5349,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",https://www.netflix.com/title/81345983,567748.0,"[{'id': 9714, 'name': 'remake'}, {'id': 12565,...",en,The Guilty,A demoted police officer assigned to a call di...,72.171,"[{'id': 89261, 'logo_path': '/pEIvnw8XMErTLclR...","[{'iso_3166_1': 'US', 'name': 'United States o...",2021-09-24,0,91.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Listen carefully.,The Guilty,6.4,1425,"[{'adult': False, 'gender': 2, 'id': 131, 'kno...","[{'adult': False, 'gender': 2, 'id': 131, 'kno..."
5350,0,"[{'id': 18, 'name': 'Drama'}]",https://amazon.com/dp/B09FDDGZNL,667969.0,"[{'id': 1691, 'name': 'dance'}, {'id': 2546, '...",en,Birds of Paradise,Two dancers at an elite ballet academy in Pari...,34.424,"[{'id': 10039, 'logo_path': '/b8w4LldQolMKiLZw...","[{'iso_3166_1': 'HU', 'name': 'Hungary'}, {'is...",2021-09-23,0,113.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Betrayal is a dance.,Birds of Paradise,6.9,95,"[{'adult': False, 'gender': 1, 'id': 2048372, ...","[{'adult': False, 'gender': 0, 'id': 1113298, ..."
5351,14000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",NaN,744275.0,"[{'id': 818, 'name': 'based on novel or book'}...",en,After We Fell,"Just as Tessa's life begins to become unglued,...",536.998,"[{'id': 6626, 'logo_path': '/A1BnMoWjzjOrjzpWi...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",2021-09-01,19000000,99.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,After We Fell,7.2,1392,"[{'adult': False, 'gender': 1, 'id': 1753914, ...","[{'adult': False, 'gender': 1, 'id': 20742, 'k..."
